In [5]:
from flask import Flask, request, render_template, jsonify, json
from decouple import config
from flask_cors import CORS
import numpy as np
import networkx as nx
import tweepy
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re
from collections import Counter
import asyncio
from concurrent.futures import ThreadPoolExecutor
import requests
import nest_asyncio
nest_asyncio.apply()

#set up twitter environment
TWITTER_AUTH =tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))
TWITTER = tweepy.API(TWITTER_AUTH)

In [55]:
def get_first_user_timeline(search,index):
    timeline = TWITTER.user_timeline(
                    id=search,
                    count =200,
                    exclude_replies = False,
                    include_rts=True,
                    tweet_mode='extended'
                    )
    index.append([user_mentions["screen_name"] for um_list in 
             [ tweet.entities["user_mentions"] for tweet in timeline]
             for user_mentions in um_list ])

def get_first_user_favorites(search,index):
    timeline = TWITTER.favorites(
                    id=search,
                    count =200,
                    exclude_replies = False,
                    include_rts=True,
                    tweet_mode='extended'
                    )
    index.append([user_mentions["screen_name"] for um_list in 
             [ tweet.entities["user_mentions"] for tweet in timeline]
             for user_mentions in um_list ])


In [ ]:
import asyncio
import requests

async def main():
    loop = asyncio.get_event_loop()
    futures = [
        loop.run_in_executor(
            None, 
            requests.get, 
            'http://example.org/'
        )
        for i in range(20)
    ]
    for response in await asyncio.gather(*futures):
        pass

loop = asyncio.get_event_loop()
loop.run_until_complete(main())